# General PowerShell Exploration
Some general things about PowerShell, useful for the everyday, and to be able to leverage more of the capabilities.

Contents:
- [Memory Kindness](#Memory-Kindness): some information about various ways that PowerShell does (and _doesn't_) use memory, garbage collection, etc.
- [PSDrives](#PSDrives): access all the things in a familiar way; variables, environment, registry, certificates, functions, etc.
    - [Some examples of drives](#Some-examples-of-drivesSome-examples-of-drives)
    - [Some examples of using drives](#Some-examples-of-using-drives)

## Memory Kindness
Depending on the way we write our code, we can:
- use the minimal amount of memory per operation, and liberate ourselves from having to ever spend time/effort on later freeing up used memory
- orrr, gobble up a bunch of memory, and then try to remember to free it up later, and then be sad when things fail as we inevitably forgot about the 107GB variable we created

### Benefits of either way
#### Use the minimal amount (be kind)
We can do things like leverage the PowerShell pipeline to iterate over all the things, only committing to physical memory anything that we must absolutely persist for later operations. Here, we might issue a command that gets some objects, and then we could directly pipe those objects to a subsequent set of cmdlets.

In this way, only one such object is ever in memory at a time. Once it reaches the end of the pipeline, the object either consumed by the final cmdlet, or emitted to the host console. That object is then "gone", and so too is [essentially] the memory it had consumed.

#### Gobble up a bunch of memory, and then try to remember...
A traditional approach may be to issue some command that gets a pile of objects, save them as the value for a variable (commit them all to physical memory), then later do something with all those juicy objects in said variable. This partly stems from us trying to get our hands/heads around all the goodness that we're creating, and a bit of comfort (already know how to do that!).

The cost comes in terms of memory consumption (not always bad), and additional effort to them manually or at least explicitly manage (maybe through code) the memory we've consumed.  A potential benefit is that we enable use cases where we can then do things most quickly with all of those objects in memory at the same time.

### Some examples that illustrate memory consumption (and avoidance)
#### Memory consumption, manual cleanup

In [1]:
## Garbage Time!
## some .NET method invocation from the GC (GarbageCollection) class to get some pwsh process memory usage info; we'll just use this here to inspect our memory conusmption
$sbGetSomeProcessInfo = {[System.GC]::GetTotalMemory($false) | Select-Object @{n="When"; e={Get-Date}}, @{n="MemUsageMB"; e={$_ / 1mb}} -OutVariable +arrProcessInfo}

## get pwsh process memory usage baseline (should start a new pwsh session, first, so all clean)
. $sbGetSomeProcessInfo

The below script needs to be able to find the current output cell; this is an easy method to get it.


When                        MemUsageMB
----                        ----------
10/26/2022 5:29:16 PM 24.5275726318359



In [2]:
## first, garbage creation (gobble up a bunch of memory, but not even for good reason)
$arrLotsaInts = 1..10mb ## <-- garbage created here!
## let's see how much gerrbage
. $sbGetSomeProcessInfo


When                        MemUsageMB
----                        ----------
10/26/2022 5:29:21 PM 330.013786315918



In [3]:
## trash! Let's see if the "traditional" pipeline consumes even more after the initial memory allocation
Measure-Command {$arrLotsaInts | ForEach-Object {if (($_ % 1mb) -eq 0) {Write-Verbose -Verbose "on item '$($_/1mb)'MB!"}}} -OutVariable tspForeachObject_withGarbage
## see memory usage
. $sbGetSomeProcessInfo

VERBOSE: on item '1'MB!
VERBOSE: on item '2'MB!
VERBOSE: on item '3'MB!
VERBOSE: on item '4'MB!
VERBOSE: on item '5'MB!
VERBOSE: on item '6'MB!
VERBOSE: on item '7'MB!
VERBOSE: on item '8'MB!
VERBOSE: on item '9'MB!
VERBOSE: on item '10'MB!

Days              : 0
Hours             : 0
Minutes           : 0
Seconds           : 50
Milliseconds      : 576
Ticks             : 505765309
TotalDays         : 0.000585376515046296
TotalHours        : 0.0140490363611111
TotalMinutes      : 0.842942181666667
TotalSeconds      : 50.5765309
TotalMilliseconds : 50576.5309


When       : 10/26/2022 5:30:16 PM
MemUsageMB : 332.734001159668




Confirmed, nothing is free -- while the pipeline adds a small bit of memory consumption, it is minimal (especially compared to the amount we may have wasted on just assigning the values to a variable but then not taking advantage of all those objects being in memory at once.

Let's try some clean up

In [4]:
## Attempt at cleaning up -- does this doit?
Remove-Variable -Name arrLotsaInts
[System.GC]::GetTotalMemory($false) / 1mb

333.178726196289


What?! Hmm, this whole, "gotta manage that memory" is already starting to be a bit of a hassle. Oh well, back to what we've known from elsewhere:  let's have the system collect the trash

In [5]:
## well, time to collect that garbage, then see how the mem usage changes
[System.GC]::Collect()
. $sbGetSomeProcessInfo


When                       MemUsageMB
----                       ----------
10/26/2022 5:31:52 PM 9.7427978515625



Got it -- we have to do some work to free up that memory we grabbed.

#### Use the minimal amount (be kind)
M'kay - soo, we can use memory as desired, but let's see how to avoid/minimize memory usage while still achieving the same results

In [6]:
## garbage _avoidance_ -- use the pipeline!
Measure-Command {1..10mb | ForEach-Object {if (($_ % 1mb) -eq 0) {Write-Verbose -Verbose "on item '$($_/1mb)'MB!"}}} -OutVariable tspForeachObject
. $sbGetSomeProcessInfo

VERBOSE: on item '1'MB!
VERBOSE: on item '2'MB!
VERBOSE: on item '3'MB!
VERBOSE: on item '4'MB!
VERBOSE: on item '5'MB!
VERBOSE: on item '6'MB!
VERBOSE: on item '7'MB!
VERBOSE: on item '8'MB!
VERBOSE: on item '9'MB!
VERBOSE: on item '10'MB!

Days              : 0
Hours             : 0
Minutes           : 0
Seconds           : 48
Milliseconds      : 578
Ticks             : 485785964
TotalDays         : 0.000562252273148148
TotalHours        : 0.0134940545555556
TotalMinutes      : 0.809643273333333
TotalSeconds      : 48.5785964
TotalMilliseconds : 48578.5964


When       : 10/26/2022 5:32:48 PM
MemUsageMB : 24.5848007202148




Woo-hoo! It took about the same amount of time to run as before, but we prevented the overhead of hundreds of MBs of memory consumption -- go-o-o-o-o, us! 🎈

#### Benefit from intentional memory consumption
Now, back to the, "sometimes it's _good_ to leverage that memory". Here's a bit of an adjustment to the first example above that made all of the memory garbage to handle, but that actually takes advantage of that memory to then do all the things much more quickly. This uses the `.ForEach()` method of PowerShell arrays

In [7]:
## still garbage creation (intentional trash!), but for the sake of speed
Measure-Command {(1..10mb).ForEach({if (($_ % 1mb) -eq 0) {Write-Verbose -Verbose "on item '$($_/1mb)'MB!"}})} -OutVariable tspForeachMethod
. $sbGetSomeProcessInfo

VERBOSE: on item '1'MB!
VERBOSE: on item '2'MB!
VERBOSE: on item '3'MB!
VERBOSE: on item '4'MB!
VERBOSE: on item '5'MB!
VERBOSE: on item '6'MB!
VERBOSE: on item '7'MB!
VERBOSE: on item '8'MB!
VERBOSE: on item '9'MB!
VERBOSE: on item '10'MB!

Days              : 0
Hours             : 0
Minutes           : 0
Seconds           : 14
Milliseconds      : 283
Ticks             : 142832317
TotalDays         : 0.000165315181712963
TotalHours        : 0.00396756436111111
TotalMinutes      : 0.238053861666667
TotalSeconds      : 14.2832317
TotalMilliseconds : 14283.2317


When       : 10/26/2022 5:33:11 PM
MemUsageMB : 340.796295166016




Used lotsa memory again, but look at that speed boost!

In [8]:
## how much faster using ForeEach() method and lots of memory verus minimal memory and the pipeline? (percent)
New-Object -Type PSObject -Property @{PercentFasterForMethod = $tspForeachObject.TotalMilliseconds / $tspForeachMethod.TotalMilliseconds * 100}


PercentFasterForMethod
----------------------
      340.109279330671



Soo fast! So, we get to choose. Conserve resources (prevent memory consumption) and get done in sufficient time, or blaze up the resources and go fast fast fast!

## PSDrives
In PowerShell, things are accessible as "drives" to it easy to interact with them in a familiar way.  For more information, `Get-Help about_Providers`, or see the online version at Microsoft Docs [About Providers](https://docs.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_providers). "The data is presented in a consistent format that resembles a file system drive", so we can interact with it in the same way we do a filesystem. We can use drives to explore, troubleshoot, set, unset, etc. the things around us in an easy, direct way.

### Some examples of drives
There are several default PSDrives available in a PowerShell session.
- `Environment`: "environment variables are dynamically named variables that describe the
environment in which your programs run"; `Get-Help about_Environment_Provider`
- `Variables`: the way in which we refer to the objects available in our PowerShell session; `Get-Help about_Variable_Provider`
- `Registry`: you know, the Windows registry, where there are infinite settings available to mess with Windows; `Get-Help about_Registry_Provider`

To get the help topics for the rest of the built-in PowerShell providers, `Get-Help -Category HelpFile -Name *provider*`

### Some examples of using drives

In [3]:
## get the current PSDrives available in this PowerShell session
Get-PSDrive | Select-Object -First 5


Name           Used (GB)     Free (GB) Provider      Root                                CurrentLoc
                                                                                              ation
----           ---------     --------- --------      ----                                ----------
Alias                                  Alias                                                       
C                 433.85         42.53 FileSystem    C:\                                 …ills\docs
Cert                                   Certificate   \                                             
Env                                    Environment                                                 
Function                               Function                                                    



We can `Get-ChildItem` (alias of `dir`) using the "path" of the Variable PSDrive to explore current variables/values

In [5]:
## get some variables
dir variable:\ | Select-Object -First 5


Name                           Value
----                           -----
$                              variable:\
?                              True
^                              dir
args                           {}
ConfirmPreference              High



In [7]:
## get some variables, filtering variable name with wildcards in a familiar way
dir variable:\*preference* | Select-Object -First 5


Name                           Value
----                           -----
ConfirmPreference              High
DebugPreference                SilentlyContinue
ErrorActionPreference          Continue
ProgressPreference             Continue
VerbosePreference              SilentlyContinue



And, we can see what environment variables are set

In [16]:
## have we a environment variable set for a web proxy?
dir env:\http*proxy


Name                           Value
----                           -----
https_proxy                    https://myproxy.dom.com

